# Image classification

In this notebook you will apply transfer learning to classify flowers into one the the categories.
You will use a pre trained model from tensorflow hub to extend it for your purpose.

Please review last notebooke tl-practice0.ipynb for backgound on VGG16


In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
mpl.rcParams['figure.figsize'] = (14, 4)
mpl.rcParams['axes.grid'] = True

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras.utils import to_categorical
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import EarlyStopping

print(f"Tensorflow Version {tf.__version__}, Keras Vesion: {keras.__version__}")

## Load Flower Data set

Explore it here: https://www.tensorflow.org/datasets/catalog/tf_flowers

In [ ]:
## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(  "tf_flowers",
                        split=["train[:70%]", "train[:30%]"], ## Train test split
                        batch_size=-1,
                        as_supervised=True) 

num_classes = np.max([np.max(train_labels), np.max(test_labels)]) + 1

IMG_SIZE = 224

## Explore the dataset
print( f'''
train dataset shape:    {train_ds.shape}, 
each dataset shape:     {train_ds[0].shape}, 
train labels shape:     {train_labels.shape}
First 10 train labels:  {train_labels[0:10]}

test dataset shape:    {test_ds.shape}, 
test labels shape:     {test_labels.shape}
First 10 test labels:  {test_labels[0:10]}
Max classes :          {num_classes}

We will use IMG_SIZE:  {IMG_SIZE}
''' )

# Check the first image or some random image
plt.imshow(train_ds[5])
plt.grid(False)

In [ ]:
i1 = tf.image.crop_to_bounding_box(train_ds[5], 0, 0, IMG_SIZE, IMG_SIZE)
plt.imshow(i1)

In [ ]:
## Resizing images
# **** NOTE ****
# **** NOTE ****
# THIS IS THE CHANGE WE MAKE FOR CHANGING THE IMAGE SIZE
train_ds1 = tf.image.crop_to_bounding_box(train_ds, 0, 0, IMG_SIZE, IMG_SIZE)
test_ds1  = tf.image.crop_to_bounding_box(test_ds,  0, 0, IMG_SIZE, IMG_SIZE)
#train_ds1  = train_ds
#test_ds1   = test_ds

train_ds1 = tf.image.resize(train_ds1, ( IMG_SIZE, IMG_SIZE))
test_ds1  = tf.image.resize(test_ds1, (IMG_SIZE, IMG_SIZE))

In [ ]:
## Transforming labels to correct format
train_labels1 = to_categorical(train_labels, num_classes=num_classes)
test_labels1  = to_categorical(test_labels, num_classes=num_classes)

# visualize the output
# to_categorical() function converts a class vector (integers) to one-hot encoding
test_labels[0:5], test_labels1[0:5]

## Load model

Investigate loading alternate model and try - what changes do you need to make to use the following model. 


In [ ]:
## Preprocessing input
train_ds2 = preprocess_input(train_ds1)
test_ds2  = preprocess_input(test_ds1) 

## Preprocessing input => we wont use imported preprocess_input() function
#train_ds2 = train_ds1 /255.
#test_ds2  = test_ds1  /255.

In [ ]:
# Question: - why does load model starts with high accuracy than load_weights?
#

model = keras.models.load_model('tl-practice2.h5')
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10,  restore_best_weights=True)
hist=model.fit(train_ds2, train_labels1, epochs=20, validation_split=0.2, batch_size=32, callbacks=[es])

In [ ]:
model.evaluate(test_ds2, test_labels1)
# History of accuracy
# [0.494703471660614, 0.803814709186554] => 80% accuracy

In [ ]:
h= hist.history
print(h)
plt.title("Plot of loss and accuracy")
plt.plot(h['accuracy'], label='accuracy')
plt.plot(h['loss'], label='loss')
plt.plot(h['val_loss'], label='val_loss')
plt.plot(h['val_accuracy'], label='val_accuracy')
plt.legend()
model.save('tl-practice2.h5')